In [36]:
!pip install youtube-search-python python-dateutil thefuzz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 21.4 MB/s eta 0:00:0000:0100:01


In [88]:
from src import utils
import pandas as pd
from youtubesearchpython import Search
from datetime import timedelta
from thefuzz import fuzz
import numpy as np

In [2]:
big = pd.DataFrame(utils.CorpusMaker.from_excel('corpus_updated', only_annotated=False, only_30_corpus=False).tracks)
mbz = big['mbz_id'].unique()

In [16]:
every = pd.DataFrame(utils.CorpusMaker.from_excel('corpus_updated', only_annotated=False, only_30_corpus=False, keep_all_tracks=True).tracks)
brushes = every[(every['notes'].str.contains('brushes')) & (~every['mbz_id'].isin(mbz))].reset_index(drop=True)

In [12]:
from string import Template

class DeltaTemplate(Template):
    delimiter = "%"

def strfdelta(tdelta, fmt):
    d = {"D": tdelta.days}
    hours, rem = divmod(tdelta.seconds, 3600)
    minutes, seconds = divmod(rem, 60)
    d["H"] = '{:02d}'.format(hours)
    d["M"] = '{:02d}'.format(minutes)
    d["S"] = '{:02d}'.format(seconds)
    t = DeltaTemplate(fmt)
    return t.substitute(**d)

In [17]:
new_df = []

for idx, row in brushes.iterrows():
    print(idx, len(brushes))
    searchstr = ' '.join(i for i in [row['bandleader'], row['track_name'], row['album_name'], row['recording_year']])
    try:
        search = Search(searchstr, limit=100)
    except TypeError:
        continue

    if isinstance(row['recording_length_'], str):
        expected_dur = '00:' + row['recording_length_'].split('.')[0]
    else:
        expected_dur = row['recording_length_'].strftime('00:%H:%M')
    h, m, s = (int(i) for i in expected_dur.split(':'))
    expected_dur = timedelta(hours=h, minutes=m, seconds=s)

    track_res = []
    for i in range(100):
        try:
            res = search.result()['result'][i]
        except IndexError:
            break
        match = fuzz.ratio(searchstr, res['title'])
        try:
            dur = res['duration']
        except KeyError:
            continue
        if dur is None:
            continue
        
        if len(dur.split(':')) == 3:
            h, m, s = (int(i) for i in dur.split(':'))
        else:
            h, m, s = (int(i) for i in ('0', *dur.split(':')))
            
        ts = timedelta(hours=h, minutes=m, seconds=s)
        track_res.append(dict(link=res['link'], str_match=match, ts=strfdelta(ts, '%H:%M:%S'), ts_match=abs(expected_dur - ts)))
        
    best_match = pd.DataFrame(track_res).sort_values(by='ts_match').iloc[0]
    row = row.to_dict()
    row['time_signature'] = 4
    row['links']['external'][0] = best_match['link']
    row['excerpt_duration'] = best_match['ts']
    row['recording_length_'] = best_match['ts']
    row['timestamps']['start'] = '00:00:00'
    row['timestamps']['end'] = best_match['ts']
    new_df.append(row)


0 973
1 973
2 973
3 973
4 973
5 973
6 973
7 973
8 973
9 973
10 973
11 973
12 973
13 973
14 973
15 973
16 973
17 973
18 973
19 973
20 973
21 973
22 973
23 973
24 973
25 973
26 973
27 973
28 973
29 973
30 973
31 973
32 973
33 973
34 973
35 973
36 973
37 973
38 973
39 973
40 973
41 973
42 973
43 973
44 973
45 973
46 973
47 973
48 973
49 973
50 973
51 973
52 973
53 973
54 973
55 973
56 973
57 973
58 973
59 973
60 973
61 973
62 973
63 973
64 973
65 973
66 973
67 973
68 973
69 973
70 973
71 973
72 973
73 973
74 973
75 973
76 973
77 973
78 973
79 973
80 973
81 973
82 973
83 973
84 973
85 973
86 973
87 973
88 973
89 973
90 973
91 973
92 973
93 973
94 973
95 973
96 973
97 973
98 973
99 973
100 973
101 973
102 973
103 973
104 973
105 973
106 973
107 973
108 973
109 973
110 973
111 973
112 973
113 973
114 973
115 973
116 973
117 973
118 973
119 973
120 973
121 973
122 973
123 973
124 973
125 973
126 973
127 973
128 973
129 973
130 973
131 973
132 973
133 973
134 973
135 973
136 973
137 973
138 97

In [19]:
brushes_df = pd.DataFrame(new_df)

In [89]:
brushes_df['is_acceptable(Y/N)'] = 'Y'
brushes_df['youtube_link'] = brushes_df['links'].apply(pd.Series).explode('external')
brushes_df['recording_id_for_lbz'] = 'https://listenbrainz.org/player/?recording_mbids=' + brushes_df['mbz_id']
brushes_df['recording_date_estimate'] = '01/01/' + brushes_df['recording_year'] + ' 00:00:00'
brushes_df['link'] = ''
brushes_df['bassist'] = brushes_df['musicians'].apply(pd.Series)['bassist']
brushes_df['drummer'] = brushes_df['musicians'].apply(pd.Series)['drummer']
brushes_df['start_timestamp'] = brushes_df['timestamps'].apply(pd.Series)['start']
brushes_df['end_timestamp'] = brushes_df['timestamps'].apply(pd.Series)['end']
brushes_df = brushes_df.rename(columns={'album_name': 'release_title', 'track_name': 'recording_title'})
brushes_df['channel_overrides'] = ''
brushes_df['in_30_corpus'] = np.nan
brushes_df['has_annotations'] = np.nan

In [90]:
fp = fr'{utils.get_project_root()}/references/corpus_brushes.xlsx'
with pd.ExcelWriter(fp, engine='openpyxl') as writer:
    for key, df in brushes_df.groupby('bandleader'):
        cols = df.columns
        trans = pd.DataFrame(df.columns).transpose()
        trans.columns = cols
        big = pd.concat([trans, df], axis=0).reset_index(drop=True)
        big.to_excel(writer, sheet_name=f'{key}', index=False)

In [93]:
br = pd.DataFrame(utils.CorpusMaker.from_excel('corpus_brushes', only_annotated=False, only_30_corpus=False).tracks)

In [95]:
pd.to_timedelta('00:' + br['excerpt_duration']).sum()

Timedelta('3 days 05:10:55')